In [1]:
MACCS_best_cat_params={
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS',
 'depth': 10,
 'iterations': 900,
 'l2_leaf_reg': 0.2557273602325932,
 'leaf_estimation_method': 'Gradient',
 'learning_rate': 0.0477685119153238,
 'random_strength': 0.0061100684626963,
 'rsm': 0.6847953796328295,
 'score_function': 'L2',
 'early_stopping_rounds':20,
 'eval_metric':'R2'}
MACCS_best_xgb_params={'alpha': 0.0014906977558625,
 'booster': 'dart',
 'colsample_bytree': 0.8546173904005345,
 'eta': 0.0456833498124193,
 'gamma': 3.3542988420770624e-05,
 'grow_policy': 'depthwise',
 'lambda': 0.2943003047107695,
 'max_depth': 14,
 'min_child_weight': 10,
 'n_estimators': 220,
 'normalize_type': 'forest',
 'rate_drop': 0.024216902211587,
 'sample_type': 'weighted',
 'skip_drop': 0.001261004949685,
 'subsample': 0.7218018660663547,
 'random_state':42,
 'verbosity': 0,
 'objective': 'reg:squarederror'}
MD_best_xgb_params={'alpha': 5.965346832119681e-06,
 'booster': 'gbtree',
 'verbosity': 0,
 'tree_method': 'exact',
 'objective': 'reg:squarederror',
 'colsample_bytree': 0.7295911276220814,
 'gamma': 7.372146242868913e-08,
 'grow_policy': 'lossguide',
 'lambda': 0.0002210864159401,
 'max_depth': 9,
 'eta':0.05,
 'min_child_weight': 4,
 'n_estimators': 220,
 'normalize_type': None,
 'rate_drop':None,
 'sample_type':None,
 'skip_drop': None,
 'subsample': 0.926368451515968,
 'random_state':42}
MD_best_cat_params={'iterations': 1000, 
'learning_rate': 0.05195087349854926,
'depth': 10, 
'l2_leaf_reg': 0.8124329970418442,
'boosting_type': 'Ordered',
'bootstrap_type': 'MVS',
'random_strength': 0.00015070897262938095, 
'score_function': 'L2', 
'leaf_estimation_method':"Gradient",
'rsm': 0.3037411535337428,
'early_stopping_rounds':20,
'eval_metric':'R2'}

In [2]:
import pandas as pd
from mordred import Calculator, descriptors
from rdkit import Chem,DataStructs
from rdkit.Chem import AllChem,PandasTools
import rdkit
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, balanced_accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV, train_test_split
import xgboost as xgb
import catboost as cat
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
from rdkit.Chem import rdFingerprintGenerator

In [3]:
df=pd.read_csv(r"C:\Users\19189\Desktop\original_data.csv")
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199


In [4]:
def MACCSfp(mol):
    fp = rdkit.Chem.rdMolDescriptors.GetMACCSKeysFingerprint(mol)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [5]:
PandasTools.AddMoleculeColumnToFrame(df,'Canonical_QSARr','Molecule')
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,Molecule
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,<rdkit.Chem.rdchem.Mol object at 0x000001E59F0...
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,<rdkit.Chem.rdchem.Mol object at 0x000001E59F0...


In [6]:
df["MACCS_FPs"] = df.Molecule.apply(MACCSfp)
df.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,Molecule,MACCS_FPs
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,<rdkit.Chem.rdchem.Mol object at 0x000001E59F0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,<rdkit.Chem.rdchem.Mol object at 0x000001E59F0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
MACCS_fp_df = df["MACCS_FPs"].apply(pd.Series)
MACCS_fp_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,1,0


In [8]:
MD=pd.read_csv(r"C:\Users\19189\Desktop\mordred_descriptors.csv")
MD.head(2)

,CASRN,CATMoS_LD50_mgkg,Canonical_QSARr,LogLD50,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,68523-18-2,460.0,CC1(C)C(C1C=C(Cl)Cl)C(=O)OC(C#N)C1C=CC=C(N=1)O...,2.662758,0.0,0.0,0,0,34.992970,2.613976,...,10.344674,84.588084,416.069448,9.044988,2244,39,146.0,172.0,10.201389,6.166667
1,88-04-0,3830.0,CC1C=C(O)C=C(C)C=1Cl,3.583199,0.0,0.0,0,0,11.643052,2.307250,...,9.078065,39.748909,156.034193,8.212326,110,13,48.0,54.0,4.944444,2.222222


In [9]:
MD_x=MD.iloc[:,4:1831]
MD_x =MD_x.astype(float)
MD_x.head(2)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0.0,0.0,0.0,0.0,34.992970,2.613976,4.943050,34.992970,1.249749,4.277221,...,10.344674,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667
1,0.0,0.0,0.0,0.0,11.643052,2.307250,4.614501,11.643052,1.164305,3.206363,...,9.078065,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222


In [10]:
MACCS_x = MACCS_fp_df.iloc[:,0:167]
y = df['LogLD50']
type(y)

pandas.core.series.Series

In [11]:
data=pd.concat([MACCS_x,MD_x],axis=1)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0,0,0,0,0,0,0,0,0,...,10.344674,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667
1,0,0,0,0,0,0,0,0,0,0,...,9.078065,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222


In [12]:
data=pd.concat([data,y],axis=1)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,LogLD50
0,0,0,0,0,0,0,0,0,0,0,...,84.588084,416.069448,9.044988,2244.0,39.0,146.0,172.0,10.201389,6.166667,2.662758
1,0,0,0,0,0,0,0,0,0,0,...,39.748909,156.034193,8.212326,110.0,13.0,48.0,54.0,4.944444,2.222222,3.583199


In [13]:
y = data['LogLD50']

In [14]:
x = data.iloc[:,0:1993]

In [15]:
x_train, xtest, y_train, ytest = train_test_split(x, y, test_size=0.4,random_state=42)

In [16]:
x_test, x_val, y_test, y_val= train_test_split(xtest, ytest, test_size=0.5,random_state=42)

In [17]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
6844,0,0,0,0,0,0,0,0,0,0,...,9.820486,55.700435,373.981241,10.685178,1100.0,29.0,110.0,122.0,8.368056,4.791667
9698,0,0,0,0,0,0,0,0,0,0,...,8.286773,38.875009,159.089543,6.628731,182.0,11.0,42.0,43.0,5.472222,2.916667
10496,0,0,0,0,0,0,0,0,0,0,...,10.124509,55.316502,280.073559,8.487078,848.0,37.0,112.0,135.0,7.138889,4.611111
4754,0,0,0,0,0,0,0,0,0,0,...,9.094144,41.023148,150.104465,6.004179,150.0,14.0,52.0,58.0,5.194444,2.472222
7020,0,0,0,0,0,0,0,0,0,0,...,9.174402,43.645526,200.071464,7.695056,254.0,16.0,60.0,66.0,5.173611,3.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,0,0,0,0,0,...,9.617471,67.599297,287.199762,6.243473,1039.0,28.0,100.0,114.0,6.805556,5.055556
2919,0,0,0,0,0,0,0,0,0,0,...,8.941807,41.829266,168.151415,5.254732,211.0,13.0,54.0,58.0,4.833333,2.833333
1921,0,0,0,0,0,0,0,0,0,0,...,9.818964,60.196658,174.127726,6.004404,208.0,23.0,72.0,89.0,5.506944,2.750000
1047,0,0,0,0,0,0,0,0,0,0,...,9.620793,49.437854,313.971549,10.826605,544.0,25.0,82.0,93.0,8.006944,3.930556


In [63]:
#ccc = cat.CatBoostRegressor()
#ccc.load_model(r"C:\Users\19189\Desktop\C_MACCS.model")

In [59]:
'''
feature_importance = ccc.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(120, 60))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(x_test.iloc[:,0:167].columns)[sorted_idx])
plt.title('Feature Importance')
'''

"\nfeature_importance = ccc.feature_importances_\nsorted_idx = np.argsort(feature_importance)\nfig = plt.figure(figsize=(120, 60))\nplt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')\nplt.yticks(range(len(sorted_idx)), np.array(x_test.iloc[:,0:167].columns)[sorted_idx])\nplt.title('Feature Importance')\n"

In [60]:
#model_xgb_2 = xgb.Booster()
#model_xgb_2.load_model(r"C:\Users\19189\Desktop\X_MACCS.json")

In [61]:
#xgb.plot_importance(model_xgb_2, max_num_features=10)

In [62]:
#import graphviz
#xgb.to_graphviz(model_xgb_2)

In [18]:
models = [xgb.XGBRegressor(**MACCS_best_xgb_params),
          cat.CatBoostRegressor(**MACCS_best_cat_params),
          xgb.XGBRegressor(**MD_best_xgb_params),
          cat.CatBoostRegressor(**MD_best_cat_params)]
model_names = ['X_MACCS', 'C_MACCS','X_MD', 'C_MD']
for idx, model in enumerate(models):
    if idx<=1:
        xtest=x_test.iloc[:,0:167]
        xtrain=x_train.iloc[:,0:167]
    else:
        xtest=x_test.iloc[:,167:1993]
        xtrain=x_train.iloc[:,167:1993]
    model.fit(xtrain, y_train)
    if idx%2==0:
        file_name=r'C:\Users\19189\Desktop\{0}.json'.format(model_names[idx])
    else:
        file_name=r'C:\Users\19189\Desktop\{0}.model'.format(model_names[idx])
    model.save_model(file_name)  
    y_pred = model.predict(xtest)
    x_test[str(model_names[idx])]=y_pred
x_test['LogLD50']=y_test
x_test.head(2)  
    

0:	learn: 0.0352033	total: 314ms	remaining: 4m 41s
1:	learn: 0.0657393	total: 493ms	remaining: 3m 41s
2:	learn: 0.0961848	total: 736ms	remaining: 3m 40s
3:	learn: 0.1224760	total: 1.1s	remaining: 4m 7s
4:	learn: 0.1464854	total: 1.5s	remaining: 4m 29s
5:	learn: 0.1686772	total: 1.9s	remaining: 4m 43s
6:	learn: 0.1890763	total: 2.3s	remaining: 4m 53s
7:	learn: 0.2084652	total: 2.71s	remaining: 5m 1s
8:	learn: 0.2274397	total: 3.11s	remaining: 5m 8s
9:	learn: 0.2445457	total: 3.52s	remaining: 5m 12s
10:	learn: 0.2615273	total: 3.92s	remaining: 5m 16s
11:	learn: 0.2772594	total: 4.32s	remaining: 5m 19s
12:	learn: 0.2910569	total: 4.72s	remaining: 5m 22s
13:	learn: 0.3050003	total: 5.12s	remaining: 5m 24s
14:	learn: 0.3160546	total: 5.53s	remaining: 5m 26s
15:	learn: 0.3283579	total: 5.93s	remaining: 5m 27s
16:	learn: 0.3385533	total: 6.34s	remaining: 5m 29s
17:	learn: 0.3480551	total: 6.74s	remaining: 5m 30s
18:	learn: 0.3605057	total: 7.14s	remaining: 5m 30s
19:	learn: 0.3712587	total: 7

,0,1,2,3,4,5,6,7,8,9,...,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,C_MACCS,X_MACCS,C_MD,X_MD,LogLD50
6844,0,0,0,0,0,0,0,0,0,0,...,29.0,110.0,122.0,8.368056,4.791667,2.767417,2.365774,2.487147,2.264464,0.568202
9698,0,0,0,0,0,0,0,0,0,0,...,11.0,42.0,43.0,5.472222,2.916667,3.207148,2.953445,3.214608,3.252747,3.078819


In [19]:
df=x_test[['C_MACCS','X_MACCS','C_MD','X_MD','LogLD50']]
df.head(2)

,C_MACCS,X_MACCS,C_MD,X_MD,LogLD50
6844,2.767417,2.365774,2.487147,2.264464,0.568202
9698,3.207148,2.953445,3.214608,3.252747,3.078819


In [20]:
df.to_csv(r"C:\Users\19189\Desktop\result_xcmodel.csv",index=False)